In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# H6751 Hands-on Session 4.1 Latent Semantic Analysis

## Latent Semantic Analysis

In this notebook, we will use a toy example to illustrate how LSA work for text data. 

1. BoW features for text data
    
2. Reduced SVD for text data

3. Truncated SVD for text data

4. Using sklearn module 

## Part 1 BoW features

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
# define the corpus
documents = ["car road gas", "truck road", "car", "gas oil", "gas", "oil"]

In [4]:
bow = CountVectorizer()
sparse_fea = bow.fit_transform(documents)

In [5]:
hh = sparse_fea.todense()

In [6]:
fea_names = bow.get_feature_names()

In [7]:
# show the corpus
df_raw = pd.DataFrame(sparse_fea.todense(), columns=fea_names)
df_raw.head(10)

car  gas  oil  road  truck
0    1    1    0     1      0
1    0    0    0     1      1
2    1    0    0     0      0
3    0    1    1     0      0
4    0    1    0     0      0
5    0    0    1     0      0

#### Compute the similarity between docs "gas" and "oil"

In [8]:
vec_doc4 = sparse_fea.todense()[4]
vec_doc5 = sparse_fea.todense()[5]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(vec_doc4, vec_doc5))

[[0.]]


## Part 2 Reduced for SVD

The dimensionality of the text data can be transformed to the rank of the raw matrix. Here is the rank is 5.

In [10]:
from scipy import linalg
import numpy as np
U, s, Vh = linalg.svd(sparse_fea.todense(), full_matrices=False)

In [11]:
topic_names = ['topic1', 'topic2', 'topic3', 'topic4', 'topic5']

In [12]:
U_df = pd.DataFrame(U, columns=topic_names)
U_df.head(10)

topic1    topic2    topic3        topic4    topic5
0 -0.748623  0.286454 -0.279712 -4.440892e-16  0.528459
1 -0.279712  0.528459  0.748623  5.551115e-16 -0.286454
2 -0.203629  0.185761 -0.446563  5.773503e-01 -0.625521
3 -0.446563 -0.625521  0.203629  7.216450e-16 -0.185761
4 -0.325096 -0.219880 -0.121467 -5.773503e-01 -0.405641
5 -0.121467 -0.405641  0.325096  5.773503e-01  0.219880

In [13]:
S_df = pd.DataFrame(np.diag(s), columns=topic_names, index=topic_names)
S_df.head(10)

topic1    topic2   topic3  topic4    topic5
topic1  2.162501  0.000000  0.00000     0.0  0.000000
topic2  0.000000  1.594382  0.00000     0.0  0.000000
topic3  0.000000  0.000000  1.27529     0.0  0.000000
topic4  0.000000  0.000000  0.00000     1.0  0.000000
topic5  0.000000  0.000000  0.00000     0.0  0.393915

In [14]:
VT_df = pd.DataFrame(Vh, columns=fea_names, index=topic_names)
VT_df.head(10)

car       gas       oil          road         truck
topic1 -0.440347 -0.703020 -0.262673 -4.755303e-01 -1.293463e-01
topic2  0.296174 -0.350572 -0.646747  5.111152e-01  3.314507e-01
topic3 -0.569498 -0.154906  0.414592  3.676900e-01  5.870217e-01
topic4  0.577350 -0.577350  0.577350  4.440892e-16  3.885781e-16
topic5 -0.246402 -0.159788  0.086614  6.143584e-01 -7.271970e-01

In [15]:
V_df = pd.DataFrame(Vh.T, columns=topic_names, index=fea_names)
V_df.head(10)

topic1    topic2    topic3        topic4    topic5
car   -0.440347  0.296174 -0.569498  5.773503e-01 -0.246402
gas   -0.703020 -0.350572 -0.154906 -5.773503e-01 -0.159788
oil   -0.262673 -0.646747  0.414592  5.773503e-01  0.086614
road  -0.475530  0.511115  0.367690  4.440892e-16  0.614358
truck -0.129346  0.331451  0.587022  3.885781e-16 -0.727197

### Get the new representation for the training corpus

In [16]:
dense_matrix = U*s
RSVD_df = pd.DataFrame(dense_matrix, columns=topic_names)
RSVD_df.head(10)

topic1    topic2    topic3        topic4    topic5
0 -1.618898  0.456717 -0.356713 -4.440892e-16  0.208168
1 -0.604877  0.842566  0.954712  5.551115e-16 -0.112839
2 -0.440347  0.296174 -0.569498  5.773503e-01 -0.246402
3 -0.965693 -0.997319  0.259686  7.216450e-16 -0.073174
4 -0.703020 -0.350572 -0.154906 -5.773503e-01 -0.159788
5 -0.262673 -0.646747  0.414592  5.773503e-01  0.086614

In [17]:
# compute the similarity in the new vector space
print(cosine_similarity(dense_matrix[4].reshape(1, -1), dense_matrix[5].reshape(1, -1)))

[[7.7368667e-16]]


### Get the new representation for the new documents

In [18]:
new_docs = ["truck road oil"]
test_sparsefea = bow.transform(new_docs)
raw_testdf = pd.DataFrame(test_sparsefea.todense(), columns=fea_names)
# bow features for new_docs
print(raw_testdf.head())

   car  gas  oil  road  truck
0    0    0    1     1      1


In [19]:
test_sparsefea = bow.transform(new_docs)
test_fea       = test_sparsefea.todense() * Vh.T
# reduced svd features for new_docs
svd_testdf = pd.DataFrame(test_fea, columns=topic_names)
print(svd_testdf.head())

     topic1    topic2    topic3   topic4    topic5
0 -0.867549  0.195819  1.369303  0.57735 -0.026225


## Part 3 Truncated SVD for text data

1. set the small singular values to zero, i.e., ignore the unimportant topics
2. Try the latent dimension is 2

In [20]:
# set the singular values to zero 
s[2: ] = 0

In [21]:
new_Sdf = pd.DataFrame(np.diag(s), columns=topic_names, index=topic_names)
new_Sdf.head(10)

topic1    topic2  topic3  topic4  topic5
topic1  2.162501  0.000000     0.0     0.0     0.0
topic2  0.000000  1.594382     0.0     0.0     0.0
topic3  0.000000  0.000000     0.0     0.0     0.0
topic4  0.000000  0.000000     0.0     0.0     0.0
topic5  0.000000  0.000000     0.0     0.0     0.0

In [22]:
lowdim_dense_matrix = U*s

In [23]:
TSVD_df = pd.DataFrame(lowdim_dense_matrix, columns=topic_names)
TSVD_df.head(10)

topic1    topic2  topic3  topic4  topic5
0 -1.618898  0.456717    -0.0    -0.0     0.0
1 -0.604877  0.842566     0.0     0.0    -0.0
2 -0.440347  0.296174    -0.0     0.0    -0.0
3 -0.965693 -0.997319     0.0     0.0    -0.0
4 -0.703020 -0.350572    -0.0    -0.0    -0.0
5 -0.262673 -0.646747     0.0     0.0     0.0

In [24]:
# we can just remove the zeroos columns
TSVD_df = pd.DataFrame(lowdim_dense_matrix[:,:2], columns=topic_names[:2])
TSVD_df.head(10)

topic1    topic2
0 -1.618898  0.456717
1 -0.604877  0.842566
2 -0.440347  0.296174
3 -0.965693 -0.997319
4 -0.703020 -0.350572
5 -0.262673 -0.646747

In [25]:
# compute the new similarity
print(cosine_similarity(lowdim_dense_matrix[4].reshape(1, -1), lowdim_dense_matrix[5].reshape(1, -1)))

[[0.75020516]]


In [26]:
# assume the doc is "truck road oil"
new_docs = ["truck road oil"]
# get the bow features
test_sparsefea = bow.transform(new_docs)
# project into 2 topics latent space
new_fea = test_sparsefea.todense() * Vh[:2,:].transpose()
# visualize it
new_df = pd.DataFrame(new_fea, columns=topic_names[:2])
new_df.head(10)

topic1    topic2
0 -0.867549  0.195819

## Part 4 Use Sklearn Method: TruncatedSVD

In [27]:
from sklearn.decomposition import TruncatedSVD

In [28]:
svd = TruncatedSVD(n_components=2)

In [29]:
new_fea = svd.fit_transform(sparse_fea)

In [30]:
new_df = pd.DataFrame(new_fea, columns=topic_names[:2])
new_df.head(10)

topic1    topic2
0  1.618898 -0.456717
1  0.604877 -0.842566
2  0.440347 -0.296174
3  0.965693  0.997319
4  0.703020  0.350572
5  0.262673  0.646747

In [31]:
test_fea = svd.transform(test_sparsefea)

In [32]:
new_df = pd.DataFrame(test_fea, columns=topic_names[:2])
new_df.head(10)

topic1    topic2
0  0.867549 -0.195819

### Compare the results of sklearn method and scipy svd methods

In addition, the tutorial related to other topics models such as NMF and LDA could be found here: https://github.com/amueller/COMS4995-s19/blob/master/slides/aml-18-topic-models/aml-20-topic-models.ipynb
